In [1]:
# pip install defusedxml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from typing import List
import json
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI

Loading configurable_parameters

In [8]:
# load json
f = open("configurable_parameters.json")
params = json.load(f)

# call dict
params_dict = dict(params)

# Configure environment variables  

embed_model_type = params_dict['open_ai_api']['embed_api_type']
api_key = params_dict['open_ai_api']['api_key'] 
api_endpoint = params_dict['open_ai_api']['api_endpoint']
api_version = params_dict['open_ai_api']['api_version']
model_type = params_dict['open_ai_api']['gpt_model']

In [9]:
model = AzureChatOpenAI(temperature=0.1,
                        api_key=api_key,
                        api_version=api_version,
                        azure_endpoint=api_endpoint,
                        model=model_type)

In [10]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

Prompting and Querying

In [11]:
joke_query = "Tell me a joke."
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't skeletons fight each other?", punchline="They don't have the guts.")